## Text Generation - **Generating Trump Tweets**

In [1]:
import tensorflow as tf

import numpy as np
import pandas as pd

import os
import zipfile

from tensorflow.keras import Sequential, layers, losses, optimizers

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
file = '/content/archive (17).zip'

ref = zipfile.ZipFile(file)
ref.extractall()
ref.close()

In [3]:
df = pd.read_csv('trumptweets.csv')
df.head()

,id,link,content,date,retweets,favorites,mentions,hashtags,geo
0,1698308935,https://twitter.com/realDonaldTrump/status/169...,Be sure to tune in and watch Donald Trump on L...,2009-05-04 20:54:25,500,868,NaN,NaN,NaN
1,1701461182,https://twitter.com/realDonaldTrump/status/170...,Donald Trump will be appearing on The View tom...,2009-05-05 03:00:10,33,273,NaN,NaN,NaN
2,1737479987,https://twitter.com/realDonaldTrump/status/173...,Donald Trump reads Top Ten Financial Tips on L...,2009-05-08 15:38:08,12,18,NaN,NaN,NaN
3,1741160716,https://twitter.com/realDonaldTrump/status/174...,New Blog Post: Celebrity Apprentice Finale and...,2009-05-08 22:40:15,11,24,NaN,NaN,NaN
4,1773561338,https://twitter.com/realDonaldTrump/status/177...,"""My persona will never be that of a wallflower...",2009-05-12 16:07:28,1399,1965,NaN,NaN,NaN


In [4]:
tweets = df['content']

In [5]:
len(tweets)

41122

In [6]:
tweets[:10]

0    Be sure to tune in and watch Donald Trump on L...
1    Donald Trump will be appearing on The View tom...
2    Donald Trump reads Top Ten Financial Tips on L...
3    New Blog Post: Celebrity Apprentice Finale and...
4    "My persona will never be that of a wallflower...
5    Miss USA Tara Conner will not be fired - "I've...
6    Listen to an interview with Donald Trump discu...
7    "Strive for wholeness and keep your sense of w...
8    Enter the "Think Like A Champion" signed book ...
9    "When the achiever achieves, it's not a platea...
Name: content, dtype: object

In [7]:
maxlen = max([len(x.split()) for x in tweets])
maxlen

60

In [8]:
# Create corpus
corpus = []

for tweet in tweets:
  corpus.append(tweet.lower())

In [9]:
len(corpus)

41122

In [10]:
mini = corpus[:1000]

In [11]:
tokenizer = Tokenizer(num_words=10000, oov_token='<oov>')
tokenizer.fit_on_texts(mini)

In [12]:
word_index = tokenizer.word_index
total_words = len(word_index) + 1
total_words

3603

In [13]:
sequences = []

for tweet in mini:
  token_list = tokenizer.texts_to_sequences([tweet])[0]
  for i in range(1, len(token_list)):
    n_gram = token_list[:i + 1]
    sequences.append(n_gram)

In [14]:
maxlen = max([len(x) for x in sequences])
maxlen

32

In [15]:
sequences = np.array(pad_sequences(sequences, maxlen, padding='pre'))
sequences[:10]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,  14, 120],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,  14, 120,   3],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,  14, 120,   3,  94],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,  14, 120,   3,  94,  11],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         14, 120,   3,  94,  11,   5],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
  

In [16]:
sequences = sequences[:, :-1]
labels = sequences[:, -1]

In [17]:
labels = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [18]:
embedding = layers.Embedding(input_dim=total_words, output_dim=128, input_length = maxlen - 1)

In [19]:
model = Sequential([
    embedding,
    layers.LSTM(32),
    layers.Dense(16, 'relu'),
    layers.Dense(8, 'relu'),
    layers.Dense(total_words, 'softmax')
])

model.compile(loss=losses.CategoricalCrossentropy(),
              optimizer=optimizers.Adam(),
              metrics=['accuracy'])

In [20]:
es = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=5,
                             restore_best_weights=True, verbose=False)

history = model.fit(sequences, labels, epochs=100,
                    callbacks=[es])

Epoch 1/100
553/553 [==============================] - 35s 47ms/step - loss: 6.3380 - accuracy: 0.0397
Epoch 2/100
553/553 [==============================] - 6s 11ms/step - loss: 5.0733 - accuracy: 0.1021
Epoch 3/100
553/553 [==============================] - 6s 11ms/step - loss: 4.0055 - accuracy: 0.3015
Epoch 4/100
553/553 [==============================] - 7s 12ms/step - loss: 3.4158 - accuracy: 0.4109
Epoch 5/100
553/553 [==============================] - 5s 9ms/step - loss: 3.0114 - accuracy: 0.4875
Epoch 6/100
553/553 [==============================] - 4s 7ms/step - loss: 2.6183 - accuracy: 0.5475
Epoch 7/100
553/553 [==============================] - 5s 9ms/step - loss: 2.3406 - accuracy: 0.5908
Epoch 8/100
553/553 [==============================] - 6s 11ms/step - loss: 2.1457 - accuracy: 0.6229
Epoch 9/100
553/553 [==============================] - 5s 9ms/step - loss: 2.0016 - accuracy: 0.6447
Epoch 10/100
553/553 [==============================] - 3s 6ms/step - loss: 1.8934 - 

In [22]:
next_words = 50
suffixes = ['s', 'es', 'ed', 'ing', 'ly', 'er', 'est', 'ible', 'ful', 'ment', 'ity', 'tion', 'ance', 'ence', 'ize', 'ise', 'ify', 'fy', 'al', 'ary', 'ic', 'ous', 'ive', 'less', 'ness', 'ize', 'ise', 'en']

words = []
for k, v in word_index.items():
  words.append(k)

words = [word for word in words if word not in suffixes]
words[:10]

['<oov>', 'the', 'to', 'http', 'and', 'on', 'is', 'a', 'of', 'in']

In [31]:
def generate(steps=next_words, prompt=''):

  for _ in range(next_words):
    tokens = tokenizer.texts_to_sequences([prompt])[0]
    tokens = np.array(pad_sequences([tokens], maxlen-1, padding='pre'))
    pred = model.predict(tokens, verbose=False)
    idx = np.argmax(pred)
    word = ''

    if words[idx] != '<oov>':
      word = words[idx]
      prompt += " " + word
  return prompt

In [33]:
seed_text = str(input("Prompt: "))
generate(40, prompt=seed_text)

Prompt: Taxes


"Taxes looking then up only money lawrence taxes some gretawire usa 11 tough americans morning pqpfvm currently armed tele anywhere children's george award jon attack pass laughing energy hawaii ride wall counting solutions gcqjpr currently armed tele anywhere children's george award jon attack pass laughing energy hawaii ride wall counting solutions"